# JUAS Longitudinal tutorial 5: bunch splitting

This notebooks shows a bunch splitting, and generates a video of the longitudinal phase space.

In [ ]:
# Import basic libraries
import numpy as np
from scipy.constants import m_p, c, e
from scipy.interpolate import interp1d

# Import from Xsuite
import xtrack as xt
import xpart as xp
from xpart.longitudinal.rf_bucket import RFBucket

# Matplotlib and animation
import matplotlib.pyplot as plt
import matplotlib.animation as ani

# Matplotlib default parameters for figure size and font size
plt.rcParams.update({'font.size': 16})
plt.rcParams.update({'figure.figsize': (8, 6)})
plt.rcParams.update({'figure.autolayout': True})

# LaTeX printing
from IPython.display import display, Markdown

import warnings
warnings.filterwarnings('ignore')

# We will use FFMpeg for saving animations
# This requires `conda install -c conda-forge ffmpeg`
from matplotlib.animation import FFMpegFileWriter

In [ ]:
####################################
# INPUT HERE Simulation parameters #
####################################

# Machine parameters: circumference and Twiss parameters at injection point
circumference = 100 * 2 * np.pi # in [m]

inj_alpha_x = 0
inj_alpha_y = 0
inj_beta_x = 16.
inj_beta_y = 16.

# Machine parameters: tunes, gamma transition, momentum compaction factor
Qx = 6.3
Qy = 6.4
gamma_tr = 6.1
alpha_c = gamma_tr**(-2)

# Machine parameters: RF voltage, harmonic number, acceleration gradient, bending radius and phase offset
V_rf = 200e3 # in [V]
harmonic= 8

Bdot =  0         # in T/s
bending_radius = 70 # in [m]

# Beam parameters: initial kinetic energy, total intensity, transverse emittances
Ekin = 1.4e9       # in [eV]
intensity = 8e12
epsn_x = 5e-6      # in [m*rad]
epsn_y = 5e-6      # in [m*rad]

# Calculations: Lorentz gamma and beta
gamma = 1 + e * Ekin / (m_p * c**2)
beta = np.sqrt(1 - gamma**-2)


# Beam parameter: initial bunch length
bunch_length = 2.5*230e-9/8 # 20e-9  # full bunch length in [s]
sigma_z_0 = (bunch_length / 4) * beta * c  # rms bunch length in [m]

display(Markdown(rf'Initial bunch length: $\sigma_{{z, 0}}={sigma_z_0:.4G}$ m'))

# Number of sections in which the accelerator circumference is divided (keep it at 1)
n_segments = 1


####################################
# END OF INPUT                     #
####################################

# Calculations: total energy
Etot = gamma * m_p * c**2 / e
display(Markdown(rf'$\beta_{{Lorentz}} = {beta:.4f}$   $\gamma_{{Lorentz}} = {gamma:.4f}$ $E_{{total}} = {Etot/1e9:.3G}$ GeV'))

# Calculations: slippage factor and phase offset
eta = alpha_c - gamma**-2
display(Markdown(rf'$\eta = {eta:.4G}$'))

# Calculation: initial momentum and synchrotron rune
p0 = np.sqrt(gamma**2 - 1) * m_p * c
Qs = np.sqrt(np.abs(eta) * V_rf * harmonic/ (2 * np.pi * beta**2 * Etot)) # linear synchrotron tune without acceleration
display(Markdown(rf'$Q_s = {Qs}$'))

# Calculation: revolution time, energy increment per turn
turn_period = circumference / (beta * c)
energy_increment_per_turn = e * circumference * bending_radius * Bdot  # in [J]
energy_increment_per_turn_eV = energy_increment_per_turn / e  # in [eV]

display(Markdown(rf'Revolution time: $T_0 = {turn_period*1e6:.4G}$ $\mu s$'))

#
compensate_phase = True
if compensate_phase:
    phi_below = np.arcsin(bending_radius*circumference*Bdot/V_rf)
    phi_above = np.pi - phi_below
    
    lag_rf_above = np.rad2deg(phi_above)
    lag_rf_below = np.rad2deg(phi_below)
    
    if eta > 0:
        lag_rf = lag_rf_above
    else:
        lag_rf = lag_rf_below

In [ ]:
# INPUT: The total number of turns that must be simulated
num_turns_to_track = 3000

# INPUT: At which turn the bunch splitting starts and ends
turn_to_start_split = 100
turn_to_end_split = 2000

# INPUT: the initial and final RF voltages of the two cavity systems
# Note: for the phase offset computation, the RF voltage whould be non-zero, so we put a small initial value
Vrf1_start = 200.0e3
Vrf2_start = 1

Vrf1_end = 1
Vrf2_end = 4*200.0e3

# INPUT: the RF systems harmonic
harmonic_array = np.array([8, 16])

lag_rf_array = np.array([0, 180])  # in degrees for XSuite

#################################################
turn_array = np.arange(0, num_turns_to_track+1)

# We assume the RF voltage will be linearly decresed/increased
slope_rf1 = (Vrf1_end - Vrf1_start)/(turn_to_end_split - turn_to_start_split)
slope_rf2 = (Vrf2_end - Vrf2_start)/(turn_to_end_split - turn_to_start_split)


Vrf1_program = np.piecewise(turn_array, [turn_array  < turn_to_start_split,
                                         (turn_to_start_split <= turn_array) & (turn_array < turn_to_end_split),
                                         turn_to_end_split <= turn_array],
                                        [Vrf1_start,
                                         lambda x: slope_rf1 * x + (Vrf1_start - slope_rf1 * turn_to_start_split),
                                         Vrf1_end])


Vrf2_program = np.piecewise(turn_array, [turn_array  < turn_to_start_split,
                                         (turn_to_start_split <= turn_array) & (turn_array < turn_to_end_split),
                                         turn_to_end_split <= turn_array],
                                        [Vrf2_start,
                                         lambda x: slope_rf2 * x + (Vrf2_start - slope_rf2 * turn_to_start_split),
                                         Vrf2_end])

In [ ]:
fig, ax1 = plt.subplots(1, 1)

ax1.plot(turn_array, Vrf1_program/1e3, label=r'RF 1, h={}'.format(harmonic_array[0]))
ax1.plot(turn_array, Vrf2_program/1e3, label=r'RF 2, h={}'.format(harmonic_array[1]))

ax1.legend()

ax1.set_xlabel('Turn number')
ax1.set_ylabel('Cavity voltage [kV]')

fig.suptitle('RF Program used in the simulation')

### Tracking parameters

In [ ]:
num_particles_to_track = 5_000

## Accelerator map

In [ ]:
# Create the one turn transfer matrix
matrix = xt.LineSegmentMap(
            qx=Qx, qy=Qy,
            dqx=0, dqy=0,
            betx=inj_beta_x, alfx=inj_alpha_x,
            bety=inj_beta_y, alfy=inj_alpha_y,
            dx=0, dpx=0,
            dy=0, dpy=0,
            voltage_rf=[Vrf1_program[0], Vrf2_program[0]],
            frequency_rf=harmonic_array*1/turn_period,
            lag_rf=lag_rf_array,  # in degrees for XSuite
            momentum_compaction_factor=alpha_c,
            longitudinal_mode="nonlinear",
            length=circumference,
            energy_ref_increment=energy_increment_per_turn_eV,
            energy_increment=0,
            )

# Create the line
line = xt.Line(elements={'one_turn_map': matrix,})
line.particle_ref = xt.Particles(mass0=xp.PROTON_MASS_EV, gamma0=gamma, q0=1.0)

In [ ]:
tw = line.twiss()

p, matcher = xp.generate_matched_gaussian_bunch(
    line=line,
    num_particles=num_particles_to_track,
    nemitt_x=epsn_x,
    nemitt_y=epsn_y,
    sigma_z=sigma_z_0,
    return_matcher=True)

# # Add an offset to the particles longitudinal position
# # to check the effect of an injection mismatch on the bunch splitting
# p.zeta += 5

# Logger (log every ten turns)
log_every = 20
n_log = num_turns_to_track // log_every
mon = xt.ParticlesMonitor(
    start_at_turn=0,
    stop_at_turn=1,
    n_repetitions=n_log,
    repetition_period=log_every,
    num_particles=len(p.x))

In [ ]:
#########
# Track #
#########

z_range_list = []
z_sfp_list = []
z_ufp_list = []
upper_separatrix_list = []
i_separatrix_list = []

while p.at_turn[0] < num_turns_to_track:
    print(f'Turn {p.at_turn[0]}/{num_turns_to_track}, RF voltages: {line["one_turn_map"].voltage_rf}', end='\r', flush=True)

    # Track for log_every
    line.track(p, num_turns=log_every, turn_by_turn_monitor=mon)
    
    # Update RF voltages
    line['one_turn_map'].voltage_rf = [Vrf1_program[p.at_turn[0]], Vrf2_program[p.at_turn[0]]]
    p.reorganize() # (put lost particles at the end)

    # Compute the RF bucket every log step
    
    rf_bucket = RFBucket(
            circumference=circumference,
            gamma=p.gamma0[0].copy(),
            mass_kg=m_p,
            charge_coulomb=np.abs(p.q0) * e,
            alpha_array=np.atleast_1d(line['one_turn_map'].momentum_compaction_factor),
            harmonic_list=np.atleast_1d(harmonic_array),
            voltage_list=np.atleast_1d(line['one_turn_map'].voltage_rf),
            phi_offset_list=np.atleast_1d(line['one_turn_map'].lag_rf),
            p_increment=(energy_increment_per_turn_eV / p.beta0[0]) * (e / c),
    )
    
    z_range = np.linspace(rf_bucket.z_left, rf_bucket.z_right, 1000)
    z_range_list.append(z_range)
    z_sfp_list.append(np.pad(rf_bucket.z_sfp, (0, 2-np.shape(rf_bucket.z_sfp)[0]), 'constant'))
    z_ufp_list.append(np.pad(rf_bucket.z_ufp, (0, 3-np.shape(rf_bucket.z_ufp)[0]), 'constant'))
    upper_separatrix_list.append(rf_bucket.separatrix(z_range))
    i_separatrix_list.append(p.at_turn[0])

In [ ]:
# ############
# # Plotting #
# ############

z_separatrix = np.array(z_range_list)
upper_separatrix = np.array(upper_separatrix_list)
i_separatrix = np.array(i_separatrix_list)

f_sep_z = interp1d(i_separatrix, z_separatrix, axis=0,
                   bounds_error=False, fill_value='extrapolate')
f_sep_delta = interp1d(i_separatrix, upper_separatrix_list, axis=0,
                       bounds_error=False, fill_value='extrapolate')
f_z_sfp = interp1d(i_separatrix, z_sfp_list, axis=0,
                 bounds_error=False, fill_value='extrapolate')
f_z_ufp = interp1d(i_separatrix, z_ufp_list, axis=0,
                   bounds_error=False, fill_value='extrapolate')

In [ ]:
# Make movie (needed `conda install -c conda-forge ffmpeg``)
def update_plot(i_log, fig):

    i_turn = mon.at_turn[i_log, 0, 0]
    z_sep = f_sep_z(i_turn)
    z_sfp = f_z_sfp(i_turn)
    # This is hacky: at the begining, the bucket looks like a single RF system with h=8
    # As the h=16 is ramped, a new unstable fixed point appears. But at the beginning,
    # we must avoid using the single RF UFP
    if f_z_ufp(i_turn)[-1] == 0:
        z_ufp = 0
    else:
        z_ufp = f_z_ufp(i_turn)[1]
    delta_sep = f_sep_delta(i_turn)

    plt.clf()
    plt.plot(mon.zeta[i_log, :], mon.delta[i_log, :], '.', markersize=1)
    plt.plot(z_sep-z_ufp, delta_sep, color='C1', linewidth=2)
    plt.plot(z_sep-z_ufp, -delta_sep, color='C1', linewidth=2)
    
    plt.xlim(-40, 40)
    plt.ylim(-20e-3, 20e-3)
    plt.xlabel('z [m]')
    plt.ylabel(r'$\Delta p / p_0$')
    plt.title(f'Turn {i_turn} '
            #   r'$\sigma_\zeta = $' f'{sigma_z_rms[i_log]:.2f}\n'
            #   r'$\gamma_0 = $' f'{mon.gamma0[i_log, 0, 0]:.2f} '
            #   r'$\gamma_t = $' f'{gamma_tr:.2f} '
            #   r'$\phi_{\mathrm{rf}} = $' f'{phi_rf_deg:.2f}'
              )
    plt.subplots_adjust(left=0.2, top=0.82)
    plt.grid(alpha=0.5)

fig = plt.figure()

moviewriter = FFMpegFileWriter(fps=5)
with moviewriter.saving(fig, 'splitting_PS.mp4', dpi=100):
    for j in range(0, len(mon.zeta[:, 0, 0]), 1):
    # for j in range(0, 10, 1):
        print(f'Frame {j}/{len(mon.zeta[:, 0, 0])}            ', end='\r', flush=True)
        update_plot(j, fig)
        moviewriter.grab_frame()